## Задача
Попробуйте построить модель, предсказывающую пол обладателя записи голоса.

Для этого:

Разделите выборку на обучающую и тренировочную с параметрами test_size=0.3, random_state=42.

Нормализуйте признаки с помощью функции StandardScaler(). Учитывайте, что нормализация тестовой выборки производится по среднему и отклонению тренировочной, которую мы считаем репрезентативной относительно генеральной совокупности.

Обучите модель логистической регрессии на подготовленных данных.

### Пояснение
Тестовые данные не должны влиять на параметры нормализации. Нужно использовать SCALER.TRANSFORM вместо SCALER.FIT_TRANSFORM, чтобы применять параметры нормализации, рассчитанные для тренировочных данных. Иначе данные в трейне и в тесте будут нормализованы по - разному.

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
data = pd.read_csv("voiceDataSet.csv", encoding = 'ISO-8859-1', low_memory = False)
data['label'] = (data['label'] == 'male').astype('int32')

In [6]:
y = data['label']
X = data.drop(['label'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

0.9737


In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

Вычислите значение метрики  и введите полученное значение. Запишите ответ в формате трех знаков после запятой (не округляя).

In [ ]:
y_pred = lr.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(np.round(accuracy, 4))

## Задача
В этом кейсе мы попробуем определять типы стекла по его характеристикам.

In [7]:
data_glass = pd.read_csv("glass.csv", encoding = 'ISO-8859-1', low_memory = False)

Откроем данные и увидим, что в первых столбцах показатели содержания различных веществ в стекле, а в последнем (Type) — непосредственно тип стекла.

In [8]:
data_glass

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0,1
...,...,...,...,...,...,...,...,...,...,...
209,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0,7
210,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0,7
211,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0,7
212,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0,7


Наша классификация мультиклассовая, поэтому мы должны посмотреть, сколько у нас классов. Если вы вообще ничего не знаете о задаче в плане количества классов, обязательно начинайте с этого, поскольку это повлияет на ход решения. Особенно важно понять, бинарная классификация или нет.

In [13]:
print(data_glass.Type.value_counts())
len(data_glass.Type.value_counts())

2    76
1    70
7    29
3    17
5    13
6     9
Name: Type, dtype: int64


6

Итак, мы выяснили, сколько у нас классов и узнали, что в этот раз у нас будет не бинарная классификация. А значит, мы не сможем использовать некоторые метрики качества (например,  и ).

Приступим непосредственно к построению модели. На примере этой задачи мы узнаем новый алгоритм kNN (метод ближайших соседей). Это один из простейших методов классификации. 

Его называют ленивым классификатором, потому что во время обучения модели он ничего не делает, просто считывает и сохраняет тренировочные данные. Сама классификация для него начинается тогда, когда ему дают тестовые данные. Тогда kNN проходит два базовых шага:

1. Сначала он ищет k ближайших размеченных точек данных – эти точки и называют k ближайшими соседями.
2. Затем, используя классы соседей, kNN решает, как лучше классифицировать новые данные. 

In [14]:
y_g = data_glass['Type']
X_g = data_glass.drop(['Type'], axis = 1)
X_g_train, X_g_test, y_g_train, y_g_test = train_test_split(X_g, y_g, test_size = 0.3, random_state=42)

In [15]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, KFold

In [16]:
model=KNeighborsClassifier(n_neighbors=3)
kf = KFold(n_splits=5)
cross_val_score(model, X_g, y_g, cv=kf, scoring="accuracy")

array([0.44186047, 0.65116279, 0.3255814 , 0.34883721, 0.07142857])

Если мы посмотрим на результаты классификации, то увидим достаточно большой разброс значений точности, а также в целом невысокие значения.

Это произошло в силу очень маленькой выборки в условиях большого количества классов (6 классов и около 200 наблюдений — крайне мало для получения высокого качества модели). Также плохой результат возможен по причине использования довольно простого алгоритма.

### Задание
Измените количество разбиений на 10. Вычислите среднее значение метрики  по 10 разбиениям и введите ниже, округлите до сотых.

In [18]:
kf = KFold(n_splits=10)
cross_val_score(model, X_g, y_g, cv=kf, scoring="accuracy").mean()

0.5257575757575758

## Задача
Мы будем диагностировать болезни сердца по различным медицинским параметрам пациентов.

In [19]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score 

### Задание
Избавьтесь от выбросов, чтобы они не ухудшили качество нашей модели. Для того, чтобы избавиться от выбросов, уберите все данные, значения признаков которых отстают от первого или третьего квартиля более, чем на 1.5 межквартильного размаха.

In [21]:
#  создаем датасет и избавляемся от выбросов
df = pd.read_csv('heart_fin1.csv', sep = ';', decimal = '.')

def iqr_outliers(df): 
    q75, q25 = np.percentile(df, [75, 25], axis=0)
    iqr = q75 - q25
    lower_bound = q25 - 1.5 * iqr
    upper_bound = q75 + 1.5 * iqr
    return ~((df < lower_bound) | (df > upper_bound)).any(axis=1)

df1 = df[iqr_outliers(df)]

Сколько наблюдений осталось после удаления выбросов?

In [22]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284 entries, 0 to 302
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       284 non-null    int64  
 1   sex       284 non-null    int64  
 2   cp        284 non-null    int64  
 3   trestbps  284 non-null    int64  
 4   chol      284 non-null    int64  
 5   restecg   284 non-null    int64  
 6   thalach   284 non-null    int64  
 7   exang     284 non-null    int64  
 8   oldpeak   284 non-null    float64
 9   target    284 non-null    int64  
dtypes: float64(1), int64(9)
memory usage: 24.4 KB


### Задание
Теперь разбейте выборку на тестовую и обучающую с параметрами test_size=0.15, random_state=5.

Обучите модели логистической регрессии ( c параметром max_iter=1000) и KNN (с количеством соседей, равным 3) на этих данных. Вычислите метрики качества.

У какой модели выше значение ROC AUC?

In [23]:
# Разбиваем на признаки и целевую переменную
y1 = df1['target']
X1 = df1.drop('target', axis = 'columns')
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.15, random_state = 5)

In [28]:
# Тренируем модель логистической регресии и считаем auc для LR:
model_LR = LogisticRegression(max_iter=1000, random_state = 5)
model_LR.fit(X1_train, y1_train)
print(roc_auc_score(y1_test, model_LR.predict(X1_test)))

0.7651515151515152


In [29]:
# Тренируем модель KN и считаем auc для KNN:
model_KNN = KNeighborsClassifier(n_neighbors=3)
model_KNN.fit(X1_train, y1_train)
print(roc_auc_score(y1_test, model_KNN.predict(X1_test)))

0.6233766233766235


## Задача
Загрузите данные train9.csv, оставьте в данных только признаки 'fine_amount', 'state_fee', 'late_fee', 'discount_amount', 'balance_due', 'compliance', затем избавьтесь от пропусков.

Целевой переменной будет 'compliance', разделите данные на обучающую и тестовую выборки в соотношении 70%/30% без перемешивания.

Обучите DecisionTreeClassifier из scikit-learn с параметром random_state=23. Посчитайте значения метрики F1 на тренировочной выборке и на тестовой выборке. Вычтите из значения метрики на тренировочной выборки значение метрики на тестовой выборке и запишите результат в переменную result.

In [30]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [31]:
vis_data = pd.read_csv('train9.csv', encoding='ISO-8859-1', low_memory=False)

In [32]:
df_9 = vis_data[['fine_amount', 'state_fee', 'late_fee', 'discount_amount', 'balance_due', 'compliance']]

In [33]:
df_9 = df_9.dropna()

In [34]:
y9 = df_9['compliance']
X9 = df_9.drop('compliance', axis=1)

In [35]:
X9_train, X9_test, y9_train, y9_test = train_test_split(X9, y9, test_size = 0.3, random_state = 23, shuffle=False)

In [36]:
clf = DecisionTreeClassifier(random_state=23)
clf.fit(X9_train, y9_train)
y9_pred_train = clf.predict(X9_train)
f1_train = f1_score(y9_train, y9_pred_train)

In [37]:
y9_pred = clf.predict(X9_test)
f1_test = f1_score(y9_test, y9_pred)

In [38]:
result = f1_train - f1_test

In [39]:
result

0.08055341409051786